# API Request notebook for air quality satellite data from Google Earth Engine API / geemap

This notebook is used to download data from satellite instruments such as Sentinel-5P Tropomi and Terra&Aqua MODIS.

This notebook allows access to [Google Earth Engine API](https://developers.google.com/earth-engine/datasets) using [geemap](https://geemap.org/) Python library and it requests data over the region identified by the bounding box given as input.
It also calculates the mean value for each variable in the given week (defined in the **date.json** file).

**Sentinel-5P Tropomi** data considered in this notebook:
1. Nitrogen Dioxide - NO2
2. Ozone - O3
3. Sulphur Dioxide - SO2
4. Carbon Monoxide - CO
5. Formaldehyde - CH2O
6. Ultraviole Aerosol Index - UVAI

**Terra & Aqua MODIS** data considered in this notebook:
1. Aerosol Optical Depth (AOD) 550 nm 
2. Aerosol Optical Depth (AOD) 470 nm 

Two additional datasets are also downloaded in the last part of the notebook:
1. NDVI 16-Days Global
2. Soil Type OpenLandMap USDA

### Documentation

Reference material and datasets:
1. Sentinel-5P products: https://sentinels.copernicus.eu/web/sentinel/data-products
2. Terra & Aqua MODIS: https://modis.gsfc.nasa.gov/about/
3. geemap site and documentation: https://geemap.org/
4. geemap GitHub repository: https://github.com/giswqs/geemap
5. Google Earth Engine catalog: https://developers.google.com/earth-engine/datasets

Each paragraph links to the specific resource used in this notebook.

### Import libraries

In [1]:
import ee
import geemap
import os
import geopandas as gpd
import json
import ipywidgets as widgets

In [2]:
from functions import my_methods

### Authentication
A Google account is required to be able to log in and provide the requested URL:

In [3]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWieEIGmLDjB4i9Qe98Nc5erfDnfoSTydpgIGmOEmxHHR7tMdnkXnVk

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
# Set current working directory
cwd = os.getcwd()


### Bounding Box selection
The bounding box for the project is given as input and it's used as Region of Interest for the following calculations:

In [5]:
area_path = cwd + '/bounding_box/bounding_box_buffer20.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds
roi = ee.Geometry.BBox(bounds[0],bounds[1],bounds[2],bounds[3])

### Date selection
The date is provided with the following order (yyyy-mm-dd). They are stored in the **date.json** file:

In [7]:
d = open('date.json')
date = json.load(d)
year = date['year']
mais_week = date['mais_week']
rice_week = date['rice_week']
cereal_week = date['cereal_week']

In [26]:
calendar = my_methods.manuring_periods(year, mais_week, rice_week, cereal_week)

C:\Users\Administrator\anaconda3\envs\ddust\lib\site-packages\plotly_calplot\calplot.py:262: UserWarning:

Parsing '31-12-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.



In [9]:
# With this widget is possible to select from the dropdown list the required week
select_week = widgets.Dropdown(
    options=['mais_week', 'rice_week', 'cereal_week'],
    description='name:',
    disabled=False)
select_week

Dropdown(description='name:', options=('mais_week', 'rice_week', 'cereal_week'), value='mais_week')

In [10]:
# Select start and end date of the corresponding selected week:
start_date = calendar[date[select_week.value][0]][0]
end_date = calendar[date[select_week.value][-1]][-1]
print("For", select_week.value, "the starting date is", start_date,"and the ending date is" , end_date)

For mais_week the starting date is 2020-02-24 and the ending date is 2020-03-15


---

# [Google Earth Engine - NO2 - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2) 

In [12]:
#Create a map
Map_no2 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_no2

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [13]:
#Access to NO2 image collection, selecting the band name and giving a date range
no2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')\
  .select('tropospheric_NO2_column_number_density')\
  .filterDate(start_date, end_date);

#Image visualization properties
no2_viz = {
    'min': 0,
    'max': 0.0001,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
no2 = no2.mean()
Map_no2 .addLayer(no2, no2_viz, 'no2');

In [14]:
#Clip over the ROI previously defined
no2_clip = no2.clip(roi)
Map_no2.addLayer(no2_clip, no2_viz, 'no2');

In [15]:
# Set output directory
out_dir = os.path.expanduser(cwd + '/temp')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

filename = os.path.join(out_dir, 'no2.tif')

In [16]:
#Export the NO2 .tif file in the directory selected by filename
geemap.ee_export_image(no2_clip, filename=filename, scale = 1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\no2.tif


---

# [Google Earth Engine - SO2 - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2) 

In [17]:
Map_so2 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_so2 

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [18]:
#Access to SO2 image collection, selecting the band name and giving a date range
so2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2')\
  .select('SO2_column_number_density')\
  .filterDate(start_date, end_date);

#Image visualization properties
so2_viz = {
    'min': 0,
    'max': 0.001,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
so2 = so2.mean()
Map_so2 .addLayer(so2, so2_viz, 'so2');

In [19]:
#Clip over the ROI previously defined
so2_clip = so2.clip(roi)
Map_so2 .addLayer(so2_clip, so2_viz, 'so2');

In [20]:
# Set output directory for SO2
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'so2.tif')

In [21]:
geemap.ee_export_image(so2_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\so2.tif


---

# [Google Earth Engine - UV Aerosol Index - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_AER_AI) 

In [22]:
Map_uvai = geemap.Map(center=[45.5,10], zoom = 7 )
Map_uvai

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [23]:
#Access to UV Aerosol index image collection, selecting the band name and giving a date range
uvai = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI')\
  .select('absorbing_aerosol_index')\
  .filterDate(start_date, end_date);

#Image visualization properties
uvai_viz = {
    'min': -2,
    'max': 0.05,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
uvai = uvai.mean()
Map_uvai.addLayer(uvai, uvai_viz, 'uvai');

In [24]:
#Clip over the ROI previously defined
uvai_clip = uvai.clip(roi)
Map_uvai.addLayer(uvai_clip, uvai_viz, 'uvai');

In [25]:
# Set output directory for UVAI
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'uvai.tif')

In [26]:
geemap.ee_export_image(uvai_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\uvai.tif


- - -

# [Google Earth Engine - Carbon Monoxide - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO) 

In [27]:
Map_co = geemap.Map(center=[45.5,10], zoom = 7 )
Map_co

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [28]:
#Access to CO image collection, selecting the band name and giving a date range
co = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO')\
  .select('CO_column_number_density')\
  .filterDate(start_date, end_date);

#Image visualization properties
co_viz = {
    'min': 0,
    'max': 0.05,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

co = co.mean()
Map_co.addLayer(co, co_viz, 'co');

In [29]:
#Clip over the ROI previously defined
co_clip = co.clip(roi)
Map_co.addLayer(co_clip, co_viz, 'co');

In [30]:
# Set output directory for CO
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'co.tif')

In [31]:
geemap.ee_export_image(co_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\co.tif


- - -

# [Google Earth Engine - Formaldehyde - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO) 

In [32]:
Map_form = geemap.Map(center=[45.5,10], zoom = 7 )
Map_form

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [33]:
#Access to HCHO image collection, selecting the band name and giving a date range
form = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_HCHO')\
  .select('tropospheric_HCHO_column_number_density')\
  .filterDate(start_date, end_date);

#Image visualization properties
form_viz = {
    'min': 0,
    'max': 0.0003,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
form = form.mean()
Map_form.addLayer(form, form_viz, 'form');

In [34]:
#Clip over the ROI previously defined
form_clip = form.clip(roi)
Map_form.addLayer(form_clip, form_viz, 'form');

In [35]:
# Set output directory for Ozone
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'form.tif')

In [36]:
geemap.ee_export_image(form_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\form.tif


---

# [Google Earth Engine - Ozone - Sentinel-5P Tropomi](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3) 

In [37]:
Map_o3 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_o3

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [38]:
#Access to O3 image collection, selecting the band name and giving a date range
o3 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_O3')\
  .select('O3_column_number_density')\
  .filterDate(start_date, end_date);

#Image visualization properties
o3_viz = {
    'min': 0.14,
    'max': 0.17,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
o3 = o3.mean()
Map_o3.addLayer(o3, o3_viz, 'o3');

In [39]:
#Clip over the ROI previously defined
o3_clip = o3.clip(roi)
Map_o3.addLayer(o3_clip, o3_viz, 'o3');

In [40]:
# Set output directory for Ozone
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'o3.tif')

In [41]:
geemap.ee_export_image(o3_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\o3.tif


- - -

# [Google Earth Engine - AOD 550nm - Terra & Aqua MAIAC Land Aerosol Optical Depth - 1 km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD19A2_GRANULES) 

In [42]:
Map_aod55 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_aod55

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [43]:
#Access to AOD 500 nm image collection, selecting the band name and giving a date range
aod55 = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')\
  .select('Optical_Depth_055')\
  .filterDate(start_date, end_date);

#Image visualization properties
aod55_viz = {
    'min': 0,
    'max': 250,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
aod55 = aod55.mean()
Map_aod55.addLayer(aod55, aod55_viz, 'aod55');

In [44]:
#Clip over the ROI previously defined
aod55_clip = aod55.clip(roi)
Map_aod55.addLayer(aod55_clip, aod55_viz, 'aod55');

In [45]:
# Set output directory for AOD
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'aod_055.tif')

In [46]:
geemap.ee_export_image(aod55_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\aod_055.tif


- - -

# [Google Earth Engine - AOD 470nm - Terra & Aqua MAIAC Land Aerosol Optical Depth - 1 km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD19A2_GRANULES) 

In [47]:
Map_aod47 = geemap.Map(center=[45.5,10], zoom = 7 )
Map_aod47

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [48]:
#Access to AOD 470 nm image collection, selecting the band name and giving a date range
aod47 = ee.ImageCollection('MODIS/006/MCD19A2_GRANULES')\
  .select('Optical_Depth_047')\
  .filterDate(start_date, end_date);

#Image visualization properties
aod47_viz = {
    'min': 0,
    'max': 250,
    'opacity': 0.75,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
};

#Calculate the mean value for each pixel in the time range
aod47 = aod47.mean()
Map_aod47.addLayer(aod47, aod47_viz, 'aod47');

In [49]:
#Clip over the ROI previously defined
aod47_clip = aod47.clip(roi)
Map_aod47.addLayer(aod47_clip, aod47_viz, 'aod47');

In [50]:
# Set output directory for AOD
out_dir = os.path.expanduser(cwd + '/temp')

filename = os.path.join(out_dir, 'aod_047.tif')

In [51]:
geemap.ee_export_image(aod47_clip, filename=filename, scale=1000)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\aod_047.tif


---

# [Google Earth Engine - NDVI - Terra Vegetation Indices 16-Day Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1) 

In [52]:
Map_ndvi = geemap.Map(center=[45.5,10], zoom = 7 )
Map_ndvi

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [53]:
#Access to NDVI image collection, selecting the band name and giving a date range
ndvi= ee.ImageCollection('MODIS/006/MOD13Q1')\
  .select('NDVI')\
  .filterDate(start_date, end_date);

#Image visualization properties
ndvi_viz = {
    'min': 0,
    'max': 8000,
    'opacity': 0.75,
    'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
};

#Calculate the mean value for each pixel in the time range
ndvi = ndvi.mean()
Map_ndvi.addLayer(ndvi, ndvi_viz, 'ndvi');

In [54]:
ndvi_clip = ndvi.clip(roi)
Map_ndvi.addLayer(ndvi_clip, ndvi_viz, 'ndvi');

In [55]:
# Set output directory for NDVI
out_dir = os.path.expanduser(cwd + '/temp')


filename = os.path.join(out_dir, 'ndvi.tif')

In [56]:
geemap.ee_export_image(ndvi_clip, filename=filename, scale=250)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\ndvi.tif


- - -

Additional layer for Soil type classification:

# [Soil type - Open Land Map USDA](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_v02#description)

In [57]:
Map_soil = geemap.Map(center=[45.5,10], zoom = 7 )
Map_soil

Map(center=[45.5, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [58]:
#Access to USDA Open Land Map soil type image
soil = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")\
        .select('b0')

soil_viz = {
  'bands': ['b0'],
  'min': 1.0,
  'max': 12.0,
  'palette': [
    "d5c36b","b96947","9d3706","ae868f","f86714","46d143",
    "368f20","3e5a14","ffd557","fff72e","ff5a9d","ff005b",
  ]
};

In [59]:
soil_clip = soil.clip(roi)

In [60]:
Map_soil.addLayer(soil_clip, soil_viz, 'Soil Type');

In [61]:
out_dir = os.path.expanduser(cwd + '/temp')
filename = os.path.join(out_dir, 'soil_type.tif')
geemap.ee_export_image(soil_clip, filename=filename, scale=250)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\soil_type.tif
